Let's start by running schedVal on three opsim runs and then comparing the outputs

    schedulerValidation.py enigma_1189_sqlite.db --outDir enigma_1189 --seeingCol finSeeing
    schedulerValidation.py ops2_1098_sqlite.db --outDir ops2_1098 --seeingCol finSeeing
    schedulerValidation.py ops2_1093_sqlite.db --outDir ops2_1093 --seeingCol finSeeing

In [37]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles
import operator

In [66]:
# Load up all the summary stats for each run
runNames = ['enigma_1189', 'ops2_1098']#, 'ops2_1093']
summaryDictList = []
for run in runNames:
    database = db.ResultsDb(run)
    summaryList = database.getSummaryStats()
    summaryDict = {}
    for sumDict in summaryList:
        summaryDict[sumDict['metricMetadata']+','+sumDict['metricName']+','+sumDict['summaryName']] = sumDict['summaryValue']
    summaryDictList.append(summaryDict)

In [67]:
summaryDictList[0]


{u'g band, all props,RmsAngle rotSkyPos,N(-3Sigma)': 0.0,
 u'g band, DDcosmology1,Rms fiveSigmaDepth,Identity': 0.5617330067416116,
 u'r band, all props,Median fiveSigmaDepth,Median': 24.439156,
 u'y band, all props,Mean normairmass,Identity': 1.197653534001168,
 u'z band, GalacticPlane,50th%ile finSeeing,Identity': 0.8516444999999999,
 u'u band, WFD,25th%ile finSeeing,Identity': 0.708634,
 u'g band, SouthCelestialPole-18,Median fiveSigmaDepth,Identity': 24.2716285,
 u'y band, WFD,RmsAngle rotSkyPos,N(+3Sigma)': 0.0,
 u'z band, NorthEclipticSpur-18c,Mean dist2Moon,Identity': 1.2765806753585087,
 u'z band, WFD,NVisitsRatio,Min': 1.09375,
 u'i band, all props,Median finSeeing,N(+3Sigma)': 39.0,
 u'WFD only,Completeness_i,TableFraction 0.6 <= P < 0.7': 0.0,
 u'u band, all props,CoaddM5,N(+3Sigma)': 5.0,
 u'z band, DD,CoaddM5,Mean': 27.75059312421623,
 u'r band, DDcosmology1,Mean normairmass,Identity': 1.1364809452601263,
 u'g band, SouthCelestialPole-18,Count finSeeing,Identity': 8790.0,


In [68]:
class baseCompareMetric(object):
    def __init__(self, sumName=None, normVal=None, reverse=False, compareMethod=operator.div):
        """
        Set up a metric value for comparing between runs
        """
        self.normVal = normVal
        self.reverse = reverse
        self.compareMethod = compareMethod
        self.sumName = sumName
        
    def _compute(self, summaryDict):
        """
        Take the data and do some stuff
        """
        # By default, just grab the summary name
        if self.sumName is not None:
            result = summaryDict[self.sumName]
        else:
            result = 0
        
        return result
        
    def __call__(self, summaryDict, raw=False):
        """
        
        """
        result = self._compute(summaryDict)
        if self.reverse:
            result = 1./result
        if not raw:
            if self.normVal is not None:
                result = compareMethod(result,self.normVal)
        return result
        
    

In [69]:
seeing = baseCompareMetric(sumName='all band, all props,Mean finSeeing,Identity')
#seeing = baseCompareMetric()
for sumDict in summaryDictList:
    print seeing(sumDict)

0.800514499892
0.782236406054


In [70]:
class m5merge(baseCompareMetric):
    """
    
    """
    def _compute(self, summaryDict):
        filters = 'ugrizy'
        nominals=[26.1,27.4,27.5,26.8,26.1,24.9]
        result = 0
        for filterName,nominal in zip(filters,nominals):
            result += summaryDict['%s band, WFD,CoaddM5,Median' %filterName]-nominal
        return result

In [73]:
m5 = m5merge()
for sumDict in summaryDictList:
    print m5(sumDict)

-1.60230222251
-1.19605195641


So what we really need, is a list of functions that will operate on the numpy array of the restored summary metric values.  Then we make an array to hold the results that's nRuns x nFunctions in size. Then can plot any way we like!


In [7]:
def compareRuns(object):
    def __init__(self, runList, metaMetricList, dbfiles=None, path=''):
        if dbfiles is None:
            dbfiles = [os.path.join(path,run,'resultsdb.sqlite') for run in runList]
        self.resultArray = np.zeros((len(runList), len(metaMetricList)), dtype=float)
        # Loop over dbfiles and metaMetricList to fill in values for resultArray
        
    def plotCompareAll(self, **kwargs):
        """
        Label x-axis with metaMetric names, legend each line with runName
        """
    def tableCompareAll(self):
        """
        Print a table comparing all the metrics and runs
        """
    def head2headPlot(self, run1,run2):
        """
        Plot just 2 runs in a head-to-head comparision
        """